In [1]:
from datetime import datetime
from packaging import version
import itk
import os
import tensorflow as tf
import histomics_stream as hs
import time

In [2]:
use_profiler = False
print("TensorFlow version: ", tf.__version__)
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow version:  2.2.0
Found GPU at: /device:GPU:0


In [3]:
if use_profiler:
    logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs, histogram_freq=1)

In [4]:
if use_profiler and False:
    # Example code that isn't working in our case.
    summary_writer = tf.summary.create_file_writer(logs)
    with summary_writer.as_default():
        model, batched_dist, strategy = hs.readExample()
        features, metadata = hs.predictExample(model, batched_dist, strategy)
        hs.outputExample(features, metadata)

In [5]:
ticTotal = time.time()
if use_profiler:
    tf.profiler.experimental.start(logs)

In [6]:
devices, strategy, model = hs.strategy_example()
print(devices)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
strategy_example: 8.302092 seconds
['/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3', '/gpu:4', '/gpu:5', '/gpu:6', '/gpu:7']


In [7]:
tf.distribute.experimental_set_strategy(strategy)

In [8]:
with strategy.scope():
    batched_dist = hs.read_example(devices, strategy)

Image source = ['TCGA-BH-A0BZ-01Z-00-DX1.45EB3E93-A871-49C6-9EAE-90D98AE01913.svs']
all_masks = ['TCGA-BH-A0BZ-01Z-00-DX1.45EB3E93-A871-49C6-9EAE-90D98AE01913-mask.png']
read_example: 2.592753 seconds


In [9]:
with strategy.scope():
    features, metadata = hs.predict_example(model, batched_dist, strategy)

level = 0, factor = 0.5, width = 112334, height = 85047
predict_example: 81.159255 seconds


In [10]:
with strategy.scope():
    hs.output_example(features, metadata)

output_example: 7.314391 seconds


In [11]:
if use_profiler:
    tf.profiler.experimental.stop()
print('Total running time: %f seconds for %d devices' % ((time.time() - ticTotal), len(devices)))

Total running time: 99.420661 seconds for 8 devices


In [12]:
if use_profiler:
    print(logs)
    %load_ext tensorboard
    %tensorboard --logdir=logs

In [13]:
del model, batched_dist, strategy, features, metadata